# Neural nets with micrograd
This is a Notebook to the free lecture [The spelled-out intro to neural networks and backpropagation: building micrograd](https://youtu.be/VMj-3S1tku0) by [Andrey Karpathy](https://github.com/karpathy).